# Regression Models with Keras

### Instrucctions

A. Build a baseline model (5 marks)

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the
train_test_split
helper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

Submit your Jupyter Notebook with your code and comments.



B. Normalize the data (5 marks)

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?



C. Increate the number of epochs (5 marks)

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

### Installing and importing

In [1]:
# installing scikit-learn if not installed
# !pip show scikit-learn

In [2]:
# importing all the libraries needed
import pandas as pd
import numpy as np
import sklearn

## Let's download the data and read it into a pandas dataframe.

In [5]:
concrete_data = pd.read_csv('/content/drive/MyDrive/IBM_AI_Engineering_Professional_Certificate/Course 2 Introduction to Deep Learning & Neural Networks with Keras/Course_Project/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [8]:
concrete_data.shape

(1030, 9)

## 1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

***Split data into training and test sets***

In [9]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
def split_train_test(concrete_data):
  train_data, test_data = train_test_split(concrete_data, test_size=0.3, train_size=0.7, random_state=42, shuffle=True, stratify=None)
  # print(f"Training dataset is 70% = {train_data.shape}")
  # print(f"Training dataset is 30% = {test_data.shape}")
  return train_data, test_data

***Split data into predictors and target***

The target variable in this problem is the concrete sample strength and the predictors are all the other columns except age.

In [10]:
concrete_data_columns = concrete_data.columns
def split_pred_target(train_data, test_data):
  predictors_train  = train_data[concrete_data_columns[(concrete_data_columns != 'Strength') & (concrete_data_columns != 'Age')]] # all columns except Strength and Age
  target_train = train_data['Strength'] # Strength column
  # predictors_train.head()
  # target_train.head()
  predictors_test = test_data[concrete_data_columns[(concrete_data_columns != 'Strength') & (concrete_data_columns != 'Age')]] # all columns except Strength and Age
  target_test = test_data['Strength'] # Strength column
  # predictors_test.head()
  # target_test.head()
  return (predictors_train, target_train, predictors_test, target_test )


***Normalize the data***

In [11]:
def normalize(x):
  x_norm = (x - x.mean()) / x.std()
  return x_norm


def normalize_data(predictors_train, target_train, predictors_test, target_test):
  predictors_train_norm = normalize(predictors_train)
  # predictors_train_norm.head()
  target_train_norm = normalize(target_train)
  # target_train_norm.head()
  predictors_test_norm = normalize(predictors_test)
  # predictors_test_norm.head()
  target_test_norm = normalize(target_test)
  # target_test_norm.head()
  return (predictors_train_norm, target_train_norm, predictors_test_norm, target_test_norm)

In [12]:
n_cols = 7
# print(n_cols)

## 2. Train the model on the training data using 50 epochs.
## 3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

***Build a Neural Network***

Import Keras and important modules

In [13]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

Define regression model

In [14]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))  # since it has seven(7) predictors
    model.add(Dense(1))

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [15]:
# build the model
model = regression_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                80        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 91 (364.00 Byte)
Trainable params: 91 (364.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [16]:
from sklearn.metrics import mean_squared_error
mse = []

for i in range(50):
  train_data, test_data = split_train_test(concrete_data)

  predictors_train, target_train, predictors_test, target_test = split_pred_target(train_data, test_data)

  predictors_train_norm, target_train_norm, predictors_test_norm, target_test_norm = normalize_data(predictors_train, target_train, predictors_test, target_test)

  # fit the model
  model.fit(predictors_train_norm, target_train_norm, validation_split=0.3, epochs=100, verbose=2)
  prediction_test = model.predict(predictors_test_norm, batch_size=None, verbose="auto", steps=None, callbacks=None)

  mse.append(mean_squared_error(target_test_norm, prediction_test))

Streaming output truncated to the last 5000 lines.
Epoch 14/100
16/16 - 0s - loss: 0.3786 - val_loss: 0.5242 - 100ms/epoch - 6ms/step
Epoch 15/100
16/16 - 0s - loss: 0.3769 - val_loss: 0.5201 - 85ms/epoch - 5ms/step
Epoch 16/100
16/16 - 0s - loss: 0.3765 - val_loss: 0.5192 - 112ms/epoch - 7ms/step
Epoch 17/100
16/16 - 0s - loss: 0.3767 - val_loss: 0.5189 - 112ms/epoch - 7ms/step
Epoch 18/100
16/16 - 0s - loss: 0.3766 - val_loss: 0.5202 - 112ms/epoch - 7ms/step
Epoch 19/100
16/16 - 0s - loss: 0.3762 - val_loss: 0.5203 - 154ms/epoch - 10ms/step
Epoch 20/100
16/16 - 0s - loss: 0.3765 - val_loss: 0.5195 - 111ms/epoch - 7ms/step
Epoch 21/100
16/16 - 0s - loss: 0.3772 - val_loss: 0.5200 - 124ms/epoch - 8ms/step
Epoch 22/100
16/16 - 0s - loss: 0.3772 - val_loss: 0.5205 - 154ms/epoch - 10ms/step
Epoch 23/100
16/16 - 0s - loss: 0.3768 - val_loss: 0.5219 - 116ms/epoch - 7ms/step
Epoch 24/100
16/16 - 0s - loss: 0.3766 - val_loss: 0.5201 - 107ms/epoch - 7ms/step
Epoch 25/100
16/16 - 0s - loss: 0.3



---

In [18]:

print(mse)  # list of 50 mean squared errors
len(mse)

[0.6133816054158931, 0.6098669063693518, 0.6191642565433394, 0.6281830084162899, 0.634378228472151, 0.638139271480193, 0.654143821150242, 0.6562936304011511, 0.664321383103981, 0.6692730101564659, 0.6697665017450913, 0.6806087183762238, 0.678003177926894, 0.6874679033800988, 0.6916001593103317, 0.6882074323553812, 0.694057989499385, 0.6961384077520835, 0.6895213119705141, 0.6961920138242568, 0.6969858336734507, 0.7028861019473546, 0.698353300765661, 0.702055069499883, 0.6997813998000754, 0.7035195110707615, 0.7017075605888018, 0.699151418343797, 0.6976975598419847, 0.7004782451352958, 0.7022284316453832, 0.698609989596313, 0.7060110092277972, 0.7035147917021806, 0.7020235552860127, 0.7014453490732078, 0.6998714184400666, 0.7033148872317639, 0.7006950357689531, 0.692964104300947, 0.7010863676399393, 0.7008850633952758, 0.6984728567904126, 0.6929821091602799, 0.6935718494200978, 0.6940518048223108, 0.6966373861693207, 0.6970947379414796, 0.6985617131439374, 0.6962436504006954]


50

## 5. Report the mean and the standard deviation of the mean squared errors.

In [19]:
#  The  mean and the standard deviation of the mean squared errors
mean = sum(mse) / len(mse)
variance = sum([((x - mean) ** 2) for x in mse]) / len(mse)
std = variance ** 0.5

print(f'mean and the standard deviation of the mean squared errors is {mean:.5f} and {std:.5f} respectively.')


mean and the standard deviation of the mean squared errors is 0.68483 and 0.02562 respectively.


**If we compare the mean of the mean squared errors to that of part B they are bigger.**